In [33]:
# ipynb 양식으로 출력 테스트
    # 일단 2개 파일 해보자

import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    """
    주어진 PDF 파일 경로에서 전체 텍스트를 추출합니다.
    
    :param file_path: 저장된 PDF의 전체 경로
    :return: 문자열 형태의 전체 텍스트
    """
    text = ""

    try:
        doc = fitz.open(file_path)
        for page in doc:
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"[ERROR] PDF 텍스트 추출 실패: {e}")
        return ""

    return text

# text_path = "./신한_코스피200커버드콜_상품설명서.pdf"

# extract_text_from_pdf(text_path)


if __name__ == "__main__":
    text_path = "/Users/Star1/Desktop/Mankind/backend/utils/신한_투자설명서_26394호.pdf"
    
    extracted = extract_text_from_pdf(text_path)
    print(extracted)                # <- 여기를 추가
    print(f"총 {len(extracted)}자 추출됨")  # 선택사항: 길이 확인용


투 자 설 명 서
2025년 04월 04일
신한투자증권 주식회사
신한투자증권 26390호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26391호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26392호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26393호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26394호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26395호 주가연계증권(ELS) : 20,000,000,000원
신한투자증권 26396호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26397호 주가연계증권(ELS) : 10,000,000,000원
신한투자증권 26398호 주가연계증권(ELS) : 20,000,000,000원
신한투자증권 26399호 주가연계증권(ELS) : 20,000,000,000원
모집가액총액 : 130,000,000,000원
1. 증권신고의 효력발생일 :
2025년 04월 04일
2. 모집가액  :
130,000,000,000 원
3. 청약기간  :
2025년 04월 07일 ~ 2025년 04월 17일까지
4. 납입기일 :
2025년 04월 17일
5. 증권신고서 및 투자설명서의 열람장소
  가. 증권신고서 :
금융위(금감원) 전자공시시스템 →  http://dart.fss.or.kr
  나. 일괄신고 추가서류 :
금융위(금감원) 전자공시시스템 →  http://dart.fss.or.kr
  다. 투자설명서 :
금융위(금감원) 전자공시시스템 →  http://dart.fss.or.kr
서면문서 : 신한투자증권 본/지점 인쇄물
6. 안정조작 또는 시장조성에 관한 사항
해당사항 없음
이 투자설명서에 대한 증권신고의 효력발생은 정부가 증권신고서의 기재
사항이 진실 또는 정확하다는 것을 인정하거나 이 증권의 가치를 보증 또
는 승인한 것이 아니며, 이 투자설명서의 기재사항은 청약일 전에 정정될 
수 있음을 

In [32]:
# 뽑고자하는 output들 우선 뽑기

import fitz     # PyMuPDF 깔아야함
import re, sys


def extract_paragraph(text, start_pattern):
    """start_pattern이 포함된 줄부터, 빈 줄 전까지 연속된 모든 줄을 반환"""
    lines = text.splitlines()
    paragraph = []
    capture = False
    for line in lines:
        if start_pattern in line:
            capture = True
        if capture:
            # 빈 줄 만나면 종료
            if line.strip() == "":
                break
            paragraph.append(line)
    return "\n".join(paragraph).strip()

def extract_fields(pdf_path):
    doc = fitz.open(pdf_path)
    # ==== 1) 6~8p에서 단일스트링으로 뽑아낼 항목들 ====
    pages_6_8 = ""
    for idx in (5, 6, 7):  # 0-based index
            # 사람기준 6~8p 는 doc 객체 내부에서 5, 6, 7p임
        if idx < len(doc): #doc의 전체 페이지수 내에서 범위 이뤄져야함.
            pages_6_8 += doc[idx].get_text()

    results = {}

    # 1-8) 라벨 기반 단일행 추출
    labels = [
        "종목명", "기초자산"]
    
    # --그 외 정보들은 필요할 때만 사용: [, "청약기간", "납입일", "배정일", "환불일", "발행일", "만기일"]
    
    for label in labels: 
        m = re.search(rf"{label}\s*(.+)", pages_6_8)
            #f string 사용해서 label 사용. + raw string 으로 \ 사용하려고 포함
                # 1. 공백 0회이상: 탭이든 띄어쓰기든 상관없음
                # 2.  [:：]? : ASCII 콜론(:) 또는 한글·원문 콜론(：)이 있을 수도, 없을 수도(1회 or 0회)
                # 3. 다시 공백 0회 이상
                # 4. (.+) : “줄 바꿈 전까지” 임의의 문자 1회 이상을 캡처(그룹 1)
        results[label] = m.group(1).strip() if m else None

    # # 9) 고난도금융투자상품 해당여부(문구 전체)
    # m = re.search(r"\(.*?\)\s*고난도금융투자상품", pages_6_8)
    # results["고난도금융투자상품"] = m.group(0).strip() if m else None

    # # ==== 2) 로마자2. 증권의 주요 권리내용 ====
    # # (1) 상황별 손익구조
    # all_text = ""
    # for page in doc:
    #     all_text += page.get_text()
    # results["(1) 상황별 손익구조"] = extract_paragraph(all_text, "(1) 상황별 손익구조")

    # # (5) 최대이익액 및 최대손실액
    # results["(5) 최대이익액 및 최대손실액"] = extract_paragraph(all_text, "(5) 최대이익액 및 최대손실액")

    # # ==== 3) 2.권리의 내용 - (4) 투자자의 요청에 의한 중도상환 - 다. ====
    # # 중도상환 신청방법 (표 셀)
    # results["중도상환 신청방법"] = extract_paragraph(all_text, "중도상환 신청방법")

    # # 단, 중도상환 가격 산정시~ 문단 전체
    # results["단, 중도상환 가격 산정시 단락"] = extract_paragraph(all_text, "단, 중도상환 가격 산정시")

    return results

if __name__ == "__main__":

    pdf_path = "신한_투자설명서_26394호.pdf"
    data = extract_fields(pdf_path)
    # 단순 프린트
    for key, val in data.items():
        print(f"{key}: {val}")



종목명: 신한투자증권 26394호 파생결합증권(주가연계증권)
기초자산: (기초자산이 지수인 경우에는 지수의 20% 이


In [53]:
# ver2: 종목명, 기초자산, 만기일, 자동조기상환 정보 뽑기

# 1~2) 종목명, 기초자산, 만  기  일 뽑아오기
# 3~6) 자동조기상환 정보 뽑아오기

import fitz     # pip install PyMuPDF
import re

def extract_fields(pdf_path):
    doc = fitz.open(pdf_path)

    # 1) 6~8p 텍스트 합치기 (0-based 5,6,7)
    pages_6_8 = ""
    for idx in (5, 6, 7):
        if idx < doc.page_count:
            pages_6_8 += doc[idx].get_text()

    # 2) 해당 페이지 범위 안에서, '(2) 모집 또는 매출의 개요' 뒤만 남기기
    if "(2) 모집 또는 매출의 개요" in pages_6_8:
        pages_6_8 = pages_6_8.split("(2) 모집 또는 매출의 개요", 1)[1]

    results = {}
    
    labels = ["종목명", "기초자산", "만  기  일"]
    
    for label in labels: 
        m = re.search(rf"{label}\s*(.+)", pages_6_8)
            #f string 사용해서 label 사용. + raw string 으로 \ 사용하려고 포함
                # 1. 공백 0회이상: 탭이든 띄어쓰기든 상관없음
                # 2.  [:：]? : ASCII 콜론(:) 또는 한글·원문 콜론(：)이 있을 수도, 없을 수도(1회 or 0회)
                # 3. 다시 공백 0회 이상
                # 4. (.+) : “줄 바꿈 전까지” 임의의 문자 1회 이상을 캡처(그룹 1)
                
        # results[label] = m
        if label == '만  기  일':
            results['만기일'] = m.group(1).strip()
        else:
            results[label] = m.group(1).strip() if m else None

# ───────────────────────────────────────────────
     # 3) 17~21p 범위에서 '나. 자동조기상환' 섹션만 떼오기
    pages_17_21 = "".join(
        doc[p].get_text() for p in range(16, 21)
        if p < doc.page_count
    )
    # results['자동조기상환'] = pages_17_21 
    #   26394호에서 에러나옴 -> 해당 페이지는 잘 뽑힘
    
    if "나. 자동조기상환" in pages_17_21:
        section = pages_17_21.split("나. 자동조기상환", 1)[1]
    else:
        section = pages_17_21 #없으면 그냥 전체 페이지로 퉁치기
        
    # results['자동조기상환'] = section
        #   26394호에서 에러나옴 -> 해당 페이지는 잘 뽑힘

        # 섹션 끝 자를 마커
    for end_marker in ("다. 만기상환", "○ 자동조기상환평가가격"):
            #만에 하나 21p 범위 내에 '다. 만기상환'이 없다면, 예방책으로
                # '○ 자동조기상환평가가격' 을 end_marker 로 지정
        if end_marker in section:
            section = section.split(end_marker, 1)[0]
                    #->이렇게 진짜 사용하고 싶은 부분 잘라낸 것임ㅇㅇ
                    
    # results['자동조기상환'] = section
        # -> "다. 만기상환"을 end_marker로 이전까지 잘뽑힘
    
    
    # 4) 잘라낸 section에서 (차수, 조기상황발생조건 %) 뽑기
        # 예: "1차 … 최초기준가격의 90% 이상인 경우"
    triggers = re.findall(
        r"(\d차)[^\n]*?최초기준가격의\s*([\d.]+%)",
        section)


    # 5) 평가일/상환금액 section에서 (차수, 날짜, 상환금액 %) 뽑기 
    # 예: "1차 2025년 09월 25일 액면금액 × 104.65%"
    schedule_matches = re.findall(
        r"(\d차)\s+"                                   # 1) 차수 (예: “1차”)
        r"(\d{4}년\s*\d{1,2}월\s*\d{1,2}일)"           # 2) 평가일 (예: “2025년 10월 17일”)
        r"(?:\s+[^\n]+?)?"                             # 3) (선택) 조건 칸 전체를 비집고 지나감
        r"\s+액면금액\s*[×x]\s*([\d.]+%)",             # 4) 액면금액×지분% (예: “102.85%”)
        section
    )
        # schedule_matches → [("1차","2025년 10월 17일","102.85%"), ...]

    # results["자동조기상환"] = [triggers, schedule_matches]

    # 6) triggers 와 schedule_matches 를 같은 차수끼리 매칭
    auto_redemption = []
    for t_id, t_pct in triggers:
        for s_id, s_date, s_pay in schedule_matches:
            if s_id == t_id:
                auto_redemption.append([t_id, t_pct, s_date, s_pay])

    results["자동조기상환"] = auto_redemption


    # ───────────────────────────────────────────────
    # 7) "(5) 최대이익액 및 최대손실액" 섹션에서 '65%' 같은 퍼센트만 추출
    #  4~20p 텍스트 중에서
    #   → (5) 최대이익액 및 최대손실액 이후부터 → "2. 권리의 내용" 전까지
    
    # all_text = "".join(page.get_text() for page in doc)
    
    pages_5_21 = "".join(
        doc[p].get_text() for p in range(4, 21)
        if p < doc.page_count
    )

    #2가지 버전으로 나눠서 처리했음.
        #ver1은 그냥 만기일 기준으로만 % 제한이 있을 때. = '만기일가격' 임의 정의
                # 예) 만기일 기준 최초기준가격의 50%이상이어야한다
        #ver2는 만기일 기준으로도 % 제한이 있고,  ='최대손실 만기조건 비율' 임의 정의
        #   추가로 상품 전체 기간동안 '한번이라도 떨어지면 그만큼 손실나는' % 제한있을 때. ='낙인구간' 임의로 정의
                # 예) 만기일 기준 최초기준가격의 60%이상이어야 하고, 전체 기간동안 50% 미만으로 한번도 떨어지면 안된다
    
    if "(5) 최대이익액 및 최대손실액" in pages_5_21:
        max_sec = pages_5_21.split("(5) 최대이익액 및 최대손실액", 1)[1]
        if "2. 권리의 내용" in max_sec:
            max_sec = max_sec.split("2. 권리의 내용", 1)[0]

        # 7-1) ver2 판별: '최초기준가격의 XX% 미만' 패턴이 있으면 ver2
        m50 = re.search(r"최초기준가격[^%]*?([\d.]+)%\s*미만", max_sec)
        if m50:
            results["손실조건버전"] = "ver2"
            # (a) 낙폭 기준 비율
            results["낙인구간"] = m50.group(1) + "%"
            # (b) 만기조건 비율
            m75 = re.search(r"만기평가가격[^%]*?최초기준가격의\s*([\d.]+)%", max_sec)
            results["만기일가격"] = m75.group(1) + "%" if m75 else None

        else:
            # 7-2) ver1: '만기평가가격이 최초기준가격의 XX% 보다 작은' 패턴만
            m1 = re.search(r"만기평가가격이\s*최초기준가격의\s*([\d.]+)%", max_sec)
            if m1:
                results["손실조건버전"] = "ver1"
                results["만기일가격"] = m1.group(1) + "%"
            else:
                # 둘 다 못 찾으면 None 처리
                results["손실조건버전"] = None
                results["만기일가격"] = None
                results["낙인구간"] = None
                results["만기일가격"] = None
    else:
        results["손실조건버전"] = None
        results["만기일가격"] = None
        results["낙인구간"] = None
        results["만기일가격"] = None

    
    
    # if "(5) 최대이익액 및 최대손실액" in pages_5_21:
    #     max_sec = pages_5_21.split("(5) 최대이익액 및 최대손실액", 1)[1]
    #     if "2. 권리의 내용" in max_sec:
    #         max_sec = max_sec.split("2. 권리의 내용", 1)[0]
    
    #     # '최초기준가격의 65% 보다 작은' 에서 65%만 캡처
    #     m = re.search(r"만기평가가격이\s*최초기준가격의\s*([\d.]+)%", max_sec)
    #     results["만기일가격"] = m.group(1) + "%" if m else None
    # else:
    #     results["만기일가격"] = None



    return results

if __name__ == "__main__":
    pdf_path = "신한_투자설명서_26377호.pdf"  # 같은 폴더에 있으면 파일명만
    data = extract_fields(pdf_path)
    
    # print(data)
    
    for key, val in data.items():
        print(f"{key}: {val}\n")


종목명: 신한투자증권 26377호 파생결합증권(주가연계증권)

기초자산: KOSPI200 지수/S&P500 지수/EuroStoxx50 지수

만기일: 2028년 04월 17일

자동조기상환: [['1차', '90%', '2025년 10월 10일', '104.20%'], ['2차', '90%', '2026년 04월 10일', '108.40%'], ['3차', '85%', '2026년 10월 08일', '112.60%'], ['4차', '85%', '2027년 04월 09일', '116.80%'], ['5차', '80%', '2027년 10월 08일', '121.00%']]

손실조건버전: ver2

낙인구간: 50%

만기일가격: 75%

